In [1]:
import json
from enum import Enum
from pathlib import Path
import psycopg2
import ollama
import pandas as pd
from IPython.display import Image, Markdown, display
from tqdm import tqdm
import os
from dotenv import load_dotenv, find_dotenv
import urllib.parse
from langchain.messages import AIMessage
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_agent
from agentic_ai_tools import AccessAccountsReceivable, AccessPayments
import datetime as dt
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
from langchain_experimental.agents import create_pandas_dataframe_agent
from typing import Union, List, Dict
import datetime
from sqlalchemy import create_engine

In [2]:
# Langchain API KEY
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
# DATABASE Connection settings
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')

# Creating conneciton to database

engine = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DB_NAME}')
#conn = psycopg2.connect(f"dbname={DB_NAME} user={USERNAME} password={PASSWORD}")
accounts_receivables = pd.read_sql("SELECT * FROM accounts_receivable", engine)
payments= pd.read_sql("SELECT * FROM payments", engine)

In [3]:
class Context():
    """Custom runtime context schema."""
    user_id: str

In [4]:
prompt ="""
You are a senior accountant responsible for accounts receivable reconciliation.

Rules:
- When reconciliation or invoice updates are requested, you determine whether the payment is related to the invoice.
- Never ask the user follow-up questions.
- Do not explain your reasoning.
- Do not return stringified data.
- Do not return different structured data.
- DO NOT nest the payments data into the updated accounts receivable information.

Output requirements:
- Return ONLY a Python dictionary.
- The dictionary must represent the UPDATED accounts receivable.
- Do not wrap the output in text, markdown, or code fences.
"""

In [5]:
# Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.
checkpointer = InMemorySaver()

In [6]:
#tools=[AccessAccountsReceivable, AccessPayments]
model = ChatOllama(
    model="qwen3:8b",
    temperature=0,
) 
#model_with_tools = model.bind_tools(tools=tools)

In [7]:
def row_gen(dataframe):
    for _, row in dataframe.iterrows():
        yield str(row.to_dict())

In [8]:
rows_ar = row_gen(accounts_receivables)


In [9]:
test_ar = next(rows_ar)
test_ar

"{'invoice_number': 1, 'date': datetime.date(2025, 5, 1), 'customer_name': 'Planet Express', 'customer_number': 12038, 'amount': 50000.0, 'due_date': datetime.date(2025, 5, 2), 'payment': None, 'payment_date': None, 'payment_id': None}"

In [10]:
rows_p = row_gen(payments)

In [18]:
test_p = next(rows_p)
test_p

"{'transaction_id': 302947, 'payment_date': datetime.date(2025, 2, 18), 'payment_amount': 25000.0, 'payment_reference': '12038 - 1 - Second payment'}"

In [19]:
# Run agent
#`thread_id` is a unique identifier for a given conversation
config = {"configurable": {"thread_id": "38"}}
# Create the agent with default parameters
agent = create_agent(model=model, system_prompt=prompt, checkpointer=checkpointer)

# Ask the agent a question
response = agent.invoke({"messages": [{"role": "user", 
                   "content": f"""
You have been given an accounts receivable invoice {test_ar} and a received payment {test_p}. Assess whether the payment relates to the invoice. 
If not DO NOT RETURN ANYTHING. If it does, UPDATE THE ACCOUNTS RECEIVABLE DATA with the relevant information from the payment and return it to me. 
"""}]},    config=config,
    context=Context())

In [21]:
response

{'messages': [HumanMessage(content='\nYou have been given an accounts receivable invoice {\'invoice_number\': 1, \'date\': datetime.date(2025, 5, 1), \'customer_name\': \'Planet Express\', \'customer_number\': 12038, \'amount\': 50000.0, \'due_date\': datetime.date(2025, 5, 2), \'payment\': None, \'payment_date\': None, \'payment_id\': None} and a received payment {\'transaction_id\': 948347, \'payment_date\': datetime.date(2025, 1, 31), \'payment_amount\': 25000.0, \'payment_reference\': "12038 - 1 - We\'ll pay the rest later"}. Assess whether the payment relates to the invoice. \nIf not DO NOT RETURN ANYTHING. If it does, UPDATE THE ACCOUNTS RECEIVABLE DATA with the relevant information from the payment and return it to me. \n', additional_kwargs={}, response_metadata={}, id='17e675ba-2193-49e7-a778-3f7934ef996a'),
  AIMessage(content='{\n  "invoice_number": 1,\n  "date": datetime.date(2025, 5, 1),\n  "customer_name": "Planet Express",\n  "customer_number": 12038,\n  "amount": 50000.